In [1]:
#import Libraries
import warnings
warnings.filterwarnings("ignore")
import pyodbc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

In [2]:
pd.options.display.max_columns=None
pd.options.display.max_rows=None

In [3]:
X_train_data = pd.read_csv('loan_data/preprocessed_train_data.csv', index_col=0)
X_test_data = pd.read_csv('loan_data/preprocessed_test_data.csv', index_col=0)
y_train_data = pd.read_csv('loan_data/preprocessed_y_train_data.csv', index_col=0)
y_test_data = pd.read_csv('loan_data/preprocessed_y_test_data.csv', index_col=0)

In [4]:
X_train_data.head(2)

,id,member_id,loan_amnt,funded_amnt,int_rate,installment,grade,sub_grade,home_ownership,annual_inc,verification_status,loan_status,purpose,zip_code,addr_state,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,total_acc,initial_list_status,total_rec_prncp,total_rec_int,recoveries,acc_now_delinq,total_rev_hi_lim,emp_length_int,term_int,earliest_cr_line_mnths,mnths_since_issue_date,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,grade:G,sub_grade:A1,sub_grade:A2,sub_grade:A3,sub_grade:A4,sub_grade:A5,sub_grade:B1,sub_grade:B2,sub_grade:B3,sub_grade:B4,sub_grade:B5,sub_grade:C1,sub_grade:C2,sub_grade:C3,sub_grade:C4,sub_grade:C5,sub_grade:D1,sub_grade:D2,sub_grade:D3,sub_grade:D4,sub_grade:D5,sub_grade:E1,sub_grade:E2,sub_grade:E3,sub_grade:E4,sub_grade:E5,sub_grade:F1,sub_grade:F2,sub_grade:F3,sub_grade:F4,sub_grade:F5,sub_grade:G1,sub_grade:G2,sub_grade:G3,sub_grade:G4,sub_grade:G5,home_ownership:ANY,home_ownership:MORTGAGE,home_ownership:NONE,home_ownership:OTHER,home_ownership:OWN,home_ownership:RENT,verification_status:Not Verified,verification_status:Source Verified,verification_status:Verified,loan_status:Charged Off,loan_status:Current,loan_status:Default,loan_status:Does not meet the credit policy. Status:Charged Off,loan_status:Does not meet the credit policy. Status:Fully Paid,loan_status:Fully Paid,loan_status:In Grace Period,loan_status:Late (16-30 days),loan_status:Late (31-120 days),purpose:car,purpose:credit_card,purpose:debt_consolidation,purpose:educational,purpose:home_improvement,purpose:house,purpose:major_purchase,purpose:medical,purpose:moving,purpose:other,purpose:renewable_energy,purpose:small_business,purpose:vacation,purpose:wedding,addr_state:AK,addr_state:AL,addr_state:AR,addr_state:AZ,addr_state:CA,addr_state:CO,addr_state:CT,addr_state:DC,addr_state:DE,addr_state:FL,addr_state:GA,addr_state:HI,addr_state:IA,addr_state:ID,addr_state:IL,addr_state:IN,addr_state:KS,addr_state:KY,addr_state:LA,addr_state:MA,addr_state:MD,addr_state:ME,addr_state:MI,addr_state:MN,addr_state:MO,addr_state:MS,addr_state:MT,addr_state:NC,addr_state:NE,addr_state:NH,addr_state:NJ,addr_state:NM,addr_state:NV,addr_state:NY,addr_state:OH,addr_state:OK,addr_state:OR,addr_state:PA,addr_state:RI,addr_state:SC,addr_state:SD,addr_state:TN,addr_state:TX,addr_state:UT,addr_state:VA,addr_state:VT,addr_state:WA,addr_state:WI,addr_state:WV,addr_state:WY,initial_list_status:f,initial_list_status:w,home_ownership:RENT_OTHER_NONE,addr_state:ND,addr_state:ND_NE_IA_NV_FL_HI_AL,addr_state:NH_WV_WY_DC_ME,addr_state:MD_OK,addr_state:NM_AR_NC_VA_NJ_LA,addr_state:MO_RI_DE_AZ_TN_ID_MI,addr_state:OH_MA_KY_UT_PA,addr_state:MN_SD_IN_GA_WA,addr_state:OR_WI,addr_state:IL_MS_CO,addr_state:VT_CT_KS_MT_SC_AK,purpose:sm_b_educ,purpose:ren_en_mov_house_other,purpose:med_vac_wedd,purpose:home_imp_maj_pur_car,term:36,term:60,emp_length_int:0,emp_length_int:1,emp_length_int:2-4,emp_length_int:5-6,emp_length_int:7-9,emp_length_int:10,mnths_since_issue_date_factor,mnths_since_issue_date:<38,mnths_since_issue_date:38-39,mnths_since_issue_date:40-41,mnths_since_issue_date:42-48,mnths_since_issue_date:49-52,mnths_since_issue_date:53-64,mnths_since_issue_date:65-84,mnths_since_issue_date:>84,int_rate_factor,int_rate:<9.548,int_rate:9.548-12.025,int_rate:12.025-15.74,int_rate:15.74-20.281,int_rate:>20.281,funded_amnt_factor,annual_inc_factor,annual_inc:<20k,annual_inc:20k-30k,annual_inc:30k-40k,annual_inc:40k-50k,annual_inc:50k-60k,annual_inc:60k-70k,annual_inc:70k-80k,annual_inc:80k-90k,annual_inc:90k-100k,annual_inc:100k-120k,annual_inc:120k-150k,annual_inc:>150k,mths_since_last_delinq_factor,mths_since_last_delinq:<15,mths_since_last_delinq:15-37,mths_since_last_delinq:37-63,mths_since_last_delinq:>=63,mths_since_earliest_cr_line_factor,mths_since_earliest_cr_line:<140,mths_since_earliest_cr_line:140-199,mths_since_earliest_cr_line:199-270,mths_since_earliest_cr_line:270-352,mths_since_earliest_cr_line:352-434,mths_since_e

In [5]:
X_train_data.shape, X_test_data.shape, y_train_data.shape, y_test_data.shape

((373028, 270), (93257, 270), (373028, 1), (93257, 1))

### Selecting the Features

In [6]:
X_train_data.columns.values

array(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'int_rate',
       'installment', 'grade', 'sub_grade', 'home_ownership',
       'annual_inc', 'verification_status', 'loan_status', 'purpose',
       'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'inq_last_6mths',
       'mths_since_last_delinq', 'mths_since_last_record', 'open_acc',
       'pub_rec', 'total_acc', 'initial_list_status', 'total_rec_prncp',
       'total_rec_int', 'recoveries', 'acc_now_delinq',
       'total_rev_hi_lim', 'emp_length_int', 'term_int',
       'earliest_cr_line_mnths', 'mnths_since_issue_date', 'grade:A',
       'grade:B', 'grade:C', 'grade:D', 'grade:E', 'grade:F', 'grade:G',
       'sub_grade:A1', 'sub_grade:A2', 'sub_grade:A3', 'sub_grade:A4',
       'sub_grade:A5', 'sub_grade:B1', 'sub_grade:B2', 'sub_grade:B3',
       'sub_grade:B4', 'sub_grade:B5', 'sub_grade:C1', 'sub_grade:C2',
       'sub_grade:C3', 'sub_grade:C4', 'sub_grade:C5', 'sub_grade:D1',
       'sub_grade:D2', 'sub_grade:D3', 'sub_g

In [7]:
#### For our model we'll use:
X_train_with_ref_cat = X_train_data.loc[:, [
 'grade:A', 'grade:B', 'grade:C', 'grade:D', 'grade:E', 'grade:F', 'grade:G',
 'home_ownership:OWN', 'home_ownership:MORTGAGE', 'home_ownership:RENT_OTHER_NONE',
 'addr_state:NY', 'addr_state:NH_WV_WY_DC_ME', 'addr_state:MD_OK', 'addr_state:NM_AR_NC_VA_NJ_LA', 'addr_state:CA',
 'addr_state:MO_RI_DE_AZ_TN_ID_MI', 'addr_state:OH_MA_KY_UT_PA', 'addr_state:MN_SD_IN_GA_WA', 'addr_state:OR_WI',
 'addr_state:IL_MS_CO', 'addr_state:VT_CT_KS_MT_SC_AK', 'addr_state:ND_NE_IA_NV_FL_HI_AL',
 'verification_status:Source Verified', 'verification_status:Not Verified', 'verification_status:Verified',
 'purpose:ren_en_mov_house_other', 'purpose:debt_consolidation', 'purpose:med_vac_wedd', 'purpose:home_imp_maj_pur_car',
 'purpose:sm_b_educ',
 'initial_list_status:w', 'initial_list_status:f',
 'emp_length_int:1', 'emp_length_int:2-4', 'emp_length_int:5-6', 'emp_length_int:7-9', 'emp_length_int:10', 'emp_length_int:0',
 'mnths_since_issue_date:38-39', 'mnths_since_issue_date:40-41', 'mnths_since_issue_date:42-48', 'mnths_since_issue_date:49-52',
 'mnths_since_issue_date:53-64', 'mnths_since_issue_date:65-84', 'mnths_since_issue_date:>84', 'mnths_since_issue_date:<38',
 'int_rate:9.548-12.025', 'int_rate:12.025-15.74', 'int_rate:15.74-20.281', 'int_rate:>20.281', 'int_rate:<9.548',
 'annual_inc:20k-30k', 'annual_inc:30k-40k', 'annual_inc:40k-50k', 'annual_inc:50k-60k', 'annual_inc:60k-70k',
 'annual_inc:70k-80k', 'annual_inc:80k-90k', 'annual_inc:90k-100k', 'annual_inc:100k-120k', 'annual_inc:120k-150k',
 'annual_inc:>150k', 'annual_inc:<20k',
 'mths_since_last_delinq:15-37','mths_since_last_delinq:37-63', 'mths_since_last_delinq:>=63', 'mths_since_last_delinq:<15',
 'mths_since_earliest_cr_line:140-199', 'mths_since_earliest_cr_line:199-270', 'mths_since_earliest_cr_line:270-352',
 'mths_since_earliest_cr_line:352-434', 'mths_since_earliest_cr_line:>=434', 'mths_since_earliest_cr_line:<140',
 'delinq_2yrs:0', 'delinq_2yrs:1', 'delinq_2yrs:2-3', 'delinq_2yrs:>=4',
 'inq_last_6mths:0', 'inq_last_6mths:1', 'inq_last_6mths:2-3', 'inq_last_6mths:4-6', 'inq_last_6mths:>6',
 'open_acc:3-11', 'open_acc:11-20', 'open_acc:20-25', 'open_acc:25-28', 'open_acc:>=28', 'open_acc:<3',
 'pub_rec:0', 'pub_rec:1', 'pub_rec:2-4', 'pub_rec:>4',
 'acc_now_delinq:0', 'acc_now_delinq:>=1',
 'total_rev_hi_lim:5-16', 'total_rev_hi_lim:16-25', 'total_rev_hi_lim:25-36', 'total_rev_hi_lim:36-55', 
 'total_rev_hi_lim:55-73', 'total_rev_hi_lim:73-90', 'total_rev_hi_lim:>=90', 'total_rev_hi_lim:<5',
 'dti:<5.5', 'dti:5.5-8.7', 'dti:8.7-14.3', 'dti:14.3-17.5', 'dti:17.5-24.7', 'dti:24.7-35', 'dti:>35',
 'mths_since_last_record:2-41', 'mths_since_last_record:41-61', 'mths_since_last_record:61-85', 'mths_since_last_record:85-118',
 'mths_since_last_record:>=118', 'mths_since_last_record:0-2'
]]

In [8]:
#### For our model we'll use:
X_test_with_ref_cat = X_test_data.loc[:, [
 'grade:A', 'grade:B', 'grade:C', 'grade:D', 'grade:E', 'grade:F', 'grade:G',
 'home_ownership:OWN', 'home_ownership:MORTGAGE', 'home_ownership:RENT_OTHER_NONE',
 'addr_state:NY', 'addr_state:NH_WV_WY_DC_ME', 'addr_state:MD_OK', 'addr_state:NM_AR_NC_VA_NJ_LA', 'addr_state:CA',
 'addr_state:MO_RI_DE_AZ_TN_ID_MI', 'addr_state:OH_MA_KY_UT_PA', 'addr_state:MN_SD_IN_GA_WA', 'addr_state:OR_WI',
 'addr_state:IL_MS_CO', 'addr_state:VT_CT_KS_MT_SC_AK', 'addr_state:ND_NE_IA_NV_FL_HI_AL',
 'verification_status:Source Verified', 'verification_status:Not Verified', 'verification_status:Verified',
 'purpose:ren_en_mov_house_other', 'purpose:debt_consolidation', 'purpose:med_vac_wedd', 'purpose:home_imp_maj_pur_car',
 'purpose:sm_b_educ',
 'initial_list_status:w', 'initial_list_status:f',
 'emp_length_int:1', 'emp_length_int:2-4', 'emp_length_int:5-6', 'emp_length_int:7-9', 'emp_length_int:10', 'emp_length_int:0',
 'mnths_since_issue_date:38-39', 'mnths_since_issue_date:40-41', 'mnths_since_issue_date:42-48', 'mnths_since_issue_date:49-52',
 'mnths_since_issue_date:53-64', 'mnths_since_issue_date:65-84', 'mnths_since_issue_date:>84', 'mnths_since_issue_date:<38',
 'int_rate:9.548-12.025', 'int_rate:12.025-15.74', 'int_rate:15.74-20.281', 'int_rate:>20.281', 'int_rate:<9.548',
 'annual_inc:20k-30k', 'annual_inc:30k-40k', 'annual_inc:40k-50k', 'annual_inc:50k-60k', 'annual_inc:60k-70k',
 'annual_inc:70k-80k', 'annual_inc:80k-90k', 'annual_inc:90k-100k', 'annual_inc:100k-120k', 'annual_inc:120k-150k',
 'annual_inc:>150k', 'annual_inc:<20k',
 'mths_since_last_delinq:15-37','mths_since_last_delinq:37-63', 'mths_since_last_delinq:>=63', 'mths_since_last_delinq:<15',
 'mths_since_earliest_cr_line:140-199', 'mths_since_earliest_cr_line:199-270', 'mths_since_earliest_cr_line:270-352',
 'mths_since_earliest_cr_line:352-434', 'mths_since_earliest_cr_line:>=434', 'mths_since_earliest_cr_line:<140',
 'delinq_2yrs:0', 'delinq_2yrs:1', 'delinq_2yrs:2-3', 'delinq_2yrs:>=4',
 'inq_last_6mths:0', 'inq_last_6mths:1', 'inq_last_6mths:2-3', 'inq_last_6mths:4-6', 'inq_last_6mths:>6',
 'open_acc:3-11', 'open_acc:11-20', 'open_acc:20-25', 'open_acc:25-28', 'open_acc:>=28', 'open_acc:<3',
 'pub_rec:0', 'pub_rec:1', 'pub_rec:2-4', 'pub_rec:>4',
 'acc_now_delinq:0', 'acc_now_delinq:>=1',
 'total_rev_hi_lim:5-16', 'total_rev_hi_lim:16-25', 'total_rev_hi_lim:25-36', 'total_rev_hi_lim:36-55', 
 'total_rev_hi_lim:55-73', 'total_rev_hi_lim:73-90', 'total_rev_hi_lim:>=90', 'total_rev_hi_lim:<5',
 'dti:<5.5', 'dti:5.5-8.7', 'dti:8.7-14.3', 'dti:14.3-17.5', 'dti:17.5-24.7', 'dti:24.7-35', 'dti:>35',
 'mths_since_last_record:2-41', 'mths_since_last_record:41-61', 'mths_since_last_record:61-85', 'mths_since_last_record:85-118',
 'mths_since_last_record:>=118', 'mths_since_last_record:0-2'
]]

In [9]:
reference_categories = [
 'grade:G', 'home_ownership:RENT_OTHER_NONE', 'addr_state:ND_NE_IA_NV_FL_HI_AL','verification_status:Verified', 
 'purpose:sm_b_educ', 'initial_list_status:f', 'emp_length_int:0', 'mnths_since_issue_date:<38', 'int_rate:<9.548',
 'annual_inc:<20k', 'mths_since_last_delinq:<15', 'mths_since_earliest_cr_line:<140', 'delinq_2yrs:>=4', 'inq_last_6mths:>6',
 'open_acc:<3', 'pub_rec:>4', 'acc_now_delinq:>=1', 'total_rev_hi_lim:<5', 'dti:>35', 'mths_since_last_record:0-2']

In [10]:
#drop reference categories to avoid falling into the dummy variable trap
X_train = X_train_with_ref_cat.drop(columns=reference_categories)

In [11]:
X_test = X_test_with_ref_cat.drop(columns=reference_categories)

In [12]:
X_train.head(3)

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:OWN,home_ownership:MORTGAGE,addr_state:NY,addr_state:NH_WV_WY_DC_ME,addr_state:MD_OK,addr_state:NM_AR_NC_VA_NJ_LA,addr_state:CA,addr_state:MO_RI_DE_AZ_TN_ID_MI,addr_state:OH_MA_KY_UT_PA,addr_state:MN_SD_IN_GA_WA,addr_state:OR_WI,addr_state:IL_MS_CO,addr_state:VT_CT_KS_MT_SC_AK,verification_status:Source Verified,verification_status:Not Verified,purpose:ren_en_mov_house_other,purpose:debt_consolidation,purpose:med_vac_wedd,purpose:home_imp_maj_pur_car,initial_list_status:w,emp_length_int:1,emp_length_int:2-4,emp_length_int:5-6,emp_length_int:7-9,emp_length_int:10,mnths_since_issue_date:38-39,mnths_since_issue_date:40-41,mnths_since_issue_date:42-48,mnths_since_issue_date:49-52,mnths_since_issue_date:53-64,mnths_since_issue_date:65-84,mnths_since_issue_date:>84,int_rate:9.548-12.025,int_rate:12.025-15.74,int_rate:15.74-20.281,int_rate:>20.281,annual_inc:20k-30k,annual_inc:30k-40k,annual_inc:40k-50k,annual_inc:50k-60k,annual_inc:60k-70k,annual_inc:70k-80k,annual_inc:80k-90k,annual_inc:90k-100k,annual_inc:100k-120k,annual_inc:120k-150k,annual_inc:>150k,mths_since_last_delinq:15-37,mths_since_last_delinq:37-63,mths_since_last_delinq:>=63,mths_since_earliest_cr_line:140-199,mths_since_earliest_cr_line:199-270,mths_since_earliest_cr_line:270-352,mths_since_earliest_cr_line:352-434,mths_since_earliest_cr_line:>=434,delinq_2yrs:0,delinq_2yrs:1,delinq_2yrs:2-3,inq_last_6mths:0,inq_last_6mths:1,inq_last_6mths:2-3,inq_last_6mths:4-6,open_acc:3-11,open_acc:11-20,open_acc:20-25,open_acc:25-28,open_acc:>=28,pub_rec:0,pub_rec:1,pub_rec:2-4,acc_now_delinq:0,total_rev_hi_lim:5-16,total_rev_hi_lim:16-25,total_rev_hi_lim:25-36,total_rev_hi_lim:36-55,total_rev_hi_lim:55-73,total_rev_hi_lim:73-90,total_rev_hi_lim:>=90,dti:<5.5,dti:5.5-8.7,dti:8.7-14.3,dti:14.3-17.5,dti:17.5-24.7,dti:24.7-35,mths_since_last_record:2-41,mths_since_last_record:41-61,mths_since_last_record:61-85,mths_since_last_record:85-118,mths_since_last_record:>=118
456615,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
451541,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
394474,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0


In [13]:
y_train = y_train_data.copy()

In [14]:
y_test = y_test_data.copy()

In [15]:
len(X_train.columns.values)

95

In [16]:
X_train.columns

Index(['grade:A', 'grade:B', 'grade:C', 'grade:D', 'grade:E', 'grade:F',
       'home_ownership:OWN', 'home_ownership:MORTGAGE', 'addr_state:NY',
       'addr_state:NH_WV_WY_DC_ME', 'addr_state:MD_OK',
       'addr_state:NM_AR_NC_VA_NJ_LA', 'addr_state:CA',
       'addr_state:MO_RI_DE_AZ_TN_ID_MI', 'addr_state:OH_MA_KY_UT_PA',
       'addr_state:MN_SD_IN_GA_WA', 'addr_state:OR_WI', 'addr_state:IL_MS_CO',
       'addr_state:VT_CT_KS_MT_SC_AK', 'verification_status:Source Verified',
       'verification_status:Not Verified', 'purpose:ren_en_mov_house_other',
       'purpose:debt_consolidation', 'purpose:med_vac_wedd',
       'purpose:home_imp_maj_pur_car', 'initial_list_status:w',
       'emp_length_int:1', 'emp_length_int:2-4', 'emp_length_int:5-6',
       'emp_length_int:7-9', 'emp_length_int:10',
       'mnths_since_issue_date:38-39', 'mnths_since_issue_date:40-41',
       'mnths_since_issue_date:42-48', 'mnths_since_issue_date:49-52',
       'mnths_since_issue_date:53-64', 'mnths_sin

In [17]:
X_train.corr()

,grade:A,grade:B,grade:C,grade:D,grade:E,grade:F,home_ownership:OWN,home_ownership:MORTGAGE,addr_state:NY,addr_state:NH_WV_WY_DC_ME,addr_state:MD_OK,addr_state:NM_AR_NC_VA_NJ_LA,addr_state:CA,addr_state:MO_RI_DE_AZ_TN_ID_MI,addr_state:OH_MA_KY_UT_PA,addr_state:MN_SD_IN_GA_WA,addr_state:OR_WI,addr_state:IL_MS_CO,addr_state:VT_CT_KS_MT_SC_AK,verification_status:Source Verified,verification_status:Not Verified,purpose:ren_en_mov_house_other,purpose:debt_consolidation,purpose:med_vac_wedd,purpose:home_imp_maj_pur_car,initial_list_status:w,emp_length_int:1,emp_length_int:2-4,emp_length_int:5-6,emp_length_int:7-9,emp_length_int:10,mnths_since_issue_date:38-39,mnths_since_issue_date:40-41,mnths_since_issue_date:42-48,mnths_since_issue_date:49-52,mnths_since_issue_date:53-64,mnths_since_issue_date:65-84,mnths_since_issue_date:>84,int_rate:9.548-12.025,int_rate:12.025-15.74,int_rate:15.74-20.281,int_rate:>20.281,annual_inc:20k-30k,annual_inc:30k-40k,annual_inc:40k-50k,annual_inc:50k-60k,annual_inc:60k-70k,annual_inc:70k-80k,annual_inc:80k-90k,annual_inc:90k-100k,annual_inc:100k-120k,annual_inc:120k-150k,annual_inc:>150k,mths_since_last_delinq:15-37,mths_since_last_delinq:37-63,mths_since_last_delinq:>=63,mths_since_earliest_cr_line:140-199,mths_since_earliest_cr_line:199-270,mths_since_earliest_cr_line:270-352,mths_since_earliest_cr_line:352-434,mths_since_earliest_cr_line:>=434,delinq_2yrs:0,delinq_2yrs:1,delinq_2yrs:2-3,inq_last_6mths:0,inq_last_6mths:1,inq_last_6mths:2-3,inq_last_6mths:4-6,open_acc:3-11,open_acc:11-20,open_acc:20-25,open_acc:25-28,open_acc:>=28,pub_rec:0,pub_rec:1,pub_rec:2-4,acc_now_delinq:0,total_rev_hi_lim:5-16,total_rev_hi_lim:16-25,total_rev_hi_lim:25-36,total_rev_hi_lim:36-55,total_rev_hi_lim:55-73,total_rev_hi_lim:73-90,total_rev_hi_lim:>=90,dti:<5.5,dti:5.5-8.7,dti:8.7-14.3,dti:14.3-17.5,dti:17.5-24.7,dti:24.7-35,mths_since_last_record:2-41,mths_since_last_record:41-61,mths_since_last_record:61-85,mths_since_last_record:85-118,mths_since_last_record:>=118
grade:A,1.000000,-0.281397,-0.265078,-0.194559,-0.125897,-0.074754,-0.006156,0.065545,-0.002281,0.005823,-0.003380,-0.002557,0.005790,-0.006503,0.005403,-0.004394,0.002848,0.005837,0.002405,-0.019027,0.119380,-0.039510,-0.080371,-0.010311,0.048747,0.007864,0.004952,0.009365,0.004498,-0.006949,-0.011799,2.332059e-02,-0.021100,-0.030540,-3.300534e-02,-0.024458,0.084424,0.026291,-0.192524,-0.320206,-0.235271,-0.127943,-0.033720,-0.043648,-0.036903,-0.020118,-0.004546,0.006117,0.027397,3.108042e-02,0.041962,0.044372,0.041049,-0.059364,-0.042535,-0.024784,-0.033444,0.004428,0.033469,3.397731e-02,0.034089,0.086976,-0.058982,-0.051855,0.122048,-0.049767,-0.082130,-0.044594,-0.007975,0.008025,0.004175,-0.002054,-0.002105,0.099019,-0.088891,-0.038372,0.022281,NaN,NaN,NaN,NaN,NaN,NaN,0.005031,0.065787,0.048104,0.046847,0.005175,-0.038313,-0.081591,-0.036642,-0.045509,-0.051872,-0.052517,-0.009956
grade:B,-0.281397,1.000000,-0.390630,-0.286711,-0.185527,-0.110160,-0.001753,0.004768,-0.001985,-0.001775,-0.004630,-0.005415,0.007664,0.000655,-0.001248,-0.001797,0.003659,0.000311,0.000026,-0.036400,0.102099,-0.060837,-0.015360,-0.027058,-0.007143,-0.035210,0.002675,0.004999,0.002154,0.002736,-0.007729,-3.717958e-02,-0.028622,-0.020874,2.322740e-02,0.054574,0.016473,-0.004718,0.686837,0.024515,-0.346706,-0.188543,0.001953,0.007181,0.008893,0.001927,0.002443,0.000401,-0.000166,-4.000764e-03,-0.005007,-0.007795,-0.010804,-0.004673,0.000325,0.001747,-0.004133,0.004197,0.003394,9.355453e-03,0.010667,0.016136,-0.009214,-0.010983,0.089727,-0.029434,-0.065904,-0.038927,0.012652,-0.004886,-0.008562,-0.009112,-0.008305,0.002490,0.002818,-0.012703,0.009840,NaN,NaN,NaN,NaN,NaN,NaN,0.005386,0.002493,0.015320,0.022224,0.009688,-0.001761,-0.036762,-0.010025,-0.006866,-0.015696,0.017976,0.003589
grade:C,-0.265078,-0.390630,1.000000,-0.270084,-0.174768,-0.103772,-0.000105,-0.015218,-0.004842,-0.001660,0.001908,-0.000636,-0.004931,0.004150,-0.001484,0.005778,0.000864,-0.000719,-0.000484,0.

In [18]:
#drop values with Nan
X_train.drop(columns=['total_rev_hi_lim:5-16', 'total_rev_hi_lim:16-25',
       'total_rev_hi_lim:25-36', 'total_rev_hi_lim:36-55',
       'total_rev_hi_lim:55-73', 'total_rev_hi_lim:73-90',
       'total_rev_hi_lim:>=90'], inplace=True)

In [19]:
X_test.drop(columns=['total_rev_hi_lim:5-16', 'total_rev_hi_lim:16-25',
       'total_rev_hi_lim:25-36', 'total_rev_hi_lim:36-55',
       'total_rev_hi_lim:55-73', 'total_rev_hi_lim:73-90',
       'total_rev_hi_lim:>=90'], inplace=True)

## Modelling

### Logistic Regression

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [21]:
pd.options.display.max_rows=None

In [22]:
reg = LogisticRegression()

In [23]:
reg.fit(X_train, y_train)

LogisticRegression()

In [24]:
reg.intercept_

array([1.10759705])

In [25]:
reg.coef_

array([[ 1.22623763e+00,  9.75290100e-01,  7.64645895e-01,
         5.69783533e-01,  3.75837255e-01,  1.96655076e-01,
         8.16594074e-02,  1.19757658e-01, -6.93167833e-03,
         4.32066136e-01, -5.07386534e-02, -1.54959005e-02,
        -1.97727773e-03,  1.18492086e-02,  6.78727969e-02,
         7.75276183e-02,  1.95094641e-01,  1.96223752e-01,
         2.59815712e-01,  5.80150983e-04,  1.19180020e-01,
        -2.67205215e-02, -5.33019223e-02,  3.35798929e-02,
         2.86621744e-02,  5.61056136e-02,  1.15504617e-01,
         1.18890102e-01,  8.26146930e-02,  5.06287205e-02,
         1.21027269e-01, -1.82406242e-01, -2.82627201e-01,
        -5.00889123e-01, -6.32381589e-01, -8.80896422e-01,
        -1.16212216e+00, -1.11039667e+00, -3.24477030e-01,
        -5.86680755e-01, -7.97448601e-01, -9.12960623e-01,
        -7.17962592e-02, -1.41477424e-02,  9.31353671e-02,
         1.40372379e-01,  2.38312978e-01,  3.00589853e-01,
         3.81265883e-01,  4.05045383e-01,  4.66837481e-0

In [26]:
feature_name = X_train.columns.values

In [27]:
summary_table = pd.DataFrame(columns=['Feature name'], data=feature_name)
summary_table['Coefficients'] = np.transpose(reg.coef_)
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()

In [28]:
summary_table

,Feature name,Coefficients
0,Intercept,1.107597
1,grade:A,1.226238
2,grade:B,0.975290
3,grade:C,0.764646
4,grade:D,0.569784
5,grade:E,0.375837
6,grade:F,0.196655
7,home_ownership:OWN,0.081659
8,home_ownership:MORTGAGE,0.119758
9,addr_state:NY,-0.006932


### Build a Logistic Regression Model with P-values

In [31]:
import scipy.stats as stat
from sklearn import linear_model

class LogisticRegression_with_p_values:
    def __init__(self, *args, **kwargs):
        self.model = linear_model.LogisticRegression(*args, **kwargs)
        
    def fit(self, X, y):
        self.model.fit(X,y)
        denom = (2.0 * (1.0 + np.cosh(self.model.decision_function(X))))
        denom = np.tile(denom, (X.shape[1], 1)).T
        F_ij = np.dot((X / denom).T, X)
        cramer_Rao = np.linalg.inv(F_ij)
        sigma_estimates = np.sqrt(np.diagonal(cramer_Rao))
        z_scores = self.model.coef_[0] / sigma_estimates
        p_values = [stat.norm.sf(abs(x)) * 2 for x in z_scores]
        self.coef_ = self.model.coef_
        self.intercept_ = self.model.intercept_
        self.p_values = p_values

In [32]:
reg = LogisticRegression_with_p_values()

In [33]:
reg.fit(X_train, y_train)

In [34]:
feature_name = X_train.columns.values

In [35]:
summary_table = pd.DataFrame(columns=['Feature name'], data=feature_name)
summary_table['Coefficients'] = np.transpose(reg.coef_)
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()

In [36]:
p_values = reg.p_values
p_values = np.append(np.nan, np.array(p_values))
summary_table['p_values'] = p_values

In [37]:
summary_table

,Feature name,Coefficients,p_values
0,Intercept,1.107597,NaN
1,grade:A,1.226238,2.356212e-42
2,grade:B,0.975290,6.114488e-59
3,grade:C,0.764646,3.148650e-42
4,grade:D,0.569784,1.488821e-26
5,grade:E,0.375837,5.047331e-15
6,grade:F,0.196655,8.935535e-05
7,home_ownership:OWN,0.081659,4.467657e-05
8,home_ownership:MORTGAGE,0.119758,1.867193e-21
9,addr_state:NY,-0.006932,7.545289e-01


We'll check coefficients that are statistically significant, those that aren't will be removed. Inorder to not loose a lot of variables, if one or few of the dummy variables representing one original independent variable are statistically significant, we'll retain all dummy variables that represent that original independent variable.

If the p-value is lower than 0.05, then the variable is statistically significant

In [38]:
summary_table['is_significant'] = summary_table.apply(lambda x: 1 if x['p_values'] < 0.05 else 0, axis=1)

In [39]:
summary_table

,Feature name,Coefficients,p_values,is_significant
0,Intercept,1.107597,NaN,0
1,grade:A,1.226238,2.356212e-42,1
2,grade:B,0.975290,6.114488e-59,1
3,grade:C,0.764646,3.148650e-42,1
4,grade:D,0.569784,1.488821e-26,1
5,grade:E,0.375837,5.047331e-15,1
6,grade:F,0.196655,8.935535e-05,1
7,home_ownership:OWN,0.081659,4.467657e-05,1
8,home_ownership:MORTGAGE,0.119758,1.867193e-21,1
9,addr_state:NY,-0.006932,7.545289e-01,0


From the table above:
- All of the Grade dummy variables are statistically significant, so we'll keep it
- All of the home_ownership dummy variables are statistically significant, so we'll keep it
- 6/11 of the addr_state dummy variable is statistically significant, so we'll keep it
- 1/2 of the verification_status dummy variable is statistically significant, so we'll keep
- The initial_list_status variable is statistically significant, so we'll keep it
- All of the emp_length_int dummy variables are statistically significant, so we'll keep it
- All of the mnths_since_issue_date dummy variables are statistically significant, so we'll keep it
- All of the int_rate dummy variables are statistically significant, so we'll keep it
- 9/11 of the annual_inc dummy variable is statistically significant, so we'll keep it
- 2/3 of the mths_since_last_delinq dummy variable is statistically significant, so we'll keep it
- 3/5 of the mths_since_earliest_cr_line dummy variable is statistically significant, so we'll keep it
- None of the delinq_2yrs dummy variables are statistically significant, so we'll drop it
- All of the inq_last_6mths dummy variables are statistically significant, so we'll keep it
- None of the open_acc dummy variables are statistically significant, so we'll drop it
- None of the pub_rec dummy variables are statistically significant, so we'll drop it
- The acc_now_delinq variable is not statistically significant, so we'll drop it
- 4/6 of the dti dummy variable is statistically significant, so we'll keep it
- 2/3 of the customer_status dummy variable is statistically significant, so we'll keep it
- None of the mths_since_last_record dummy variables are statistically significant, so we'll drop it

In [40]:
summary_table['Feature name'].unique()

array(['Intercept', 'grade:A', 'grade:B', 'grade:C', 'grade:D', 'grade:E',
       'grade:F', 'home_ownership:OWN', 'home_ownership:MORTGAGE',
       'addr_state:NY', 'addr_state:NH_WV_WY_DC_ME', 'addr_state:MD_OK',
       'addr_state:NM_AR_NC_VA_NJ_LA', 'addr_state:CA',
       'addr_state:MO_RI_DE_AZ_TN_ID_MI', 'addr_state:OH_MA_KY_UT_PA',
       'addr_state:MN_SD_IN_GA_WA', 'addr_state:OR_WI',
       'addr_state:IL_MS_CO', 'addr_state:VT_CT_KS_MT_SC_AK',
       'verification_status:Source Verified',
       'verification_status:Not Verified',
       'purpose:ren_en_mov_house_other', 'purpose:debt_consolidation',
       'purpose:med_vac_wedd', 'purpose:home_imp_maj_pur_car',
       'initial_list_status:w', 'emp_length_int:1', 'emp_length_int:2-4',
       'emp_length_int:5-6', 'emp_length_int:7-9', 'emp_length_int:10',
       'mnths_since_issue_date:38-39', 'mnths_since_issue_date:40-41',
       'mnths_since_issue_date:42-48', 'mnths_since_issue_date:49-52',
       'mnths_since_issue_dat

In [41]:
#features to keep
features_to_drop = [
'delinq_2yrs:0', 'delinq_2yrs:1', 'delinq_2yrs:2-3',
'open_acc:3-11', 'open_acc:11-20', 'open_acc:20-25', 'open_acc:25-28', 'open_acc:>=28',
'pub_rec:0', 'pub_rec:1', 'pub_rec:2-4', 'acc_now_delinq:0',
'mths_since_last_record:2-41', 'mths_since_last_record:41-61',
'mths_since_last_record:61-85', 'mths_since_last_record:85-118',
'mths_since_last_record:>=118'
]

In [42]:
X_train.drop(columns=features_to_drop, inplace=True)

In [43]:
X_test.drop(columns=features_to_drop, inplace=True)

In [44]:
X_train.shape, y_train.shape

((373028, 71), (373028, 1))

In [45]:
X_test.shape, y_test.shape

((93257, 71), (93257, 1))

### Training after feature selection

In [46]:
reg = LogisticRegression_with_p_values(class_weight='balanced', random_state='42')

In [47]:
reg.fit(X_train, y_train)

ValueError: '42' cannot be used to seed a numpy.random.RandomState instance

In [ ]:
feature_name = X_train.columns.values

In [ ]:
summary_table = pd.DataFrame(columns=['Feature name'], data=feature_name)
summary_table['Coefficients'] = np.transpose(reg.coef_)
summary_table.index = summary_table.index + 1
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]
summary_table = summary_table.sort_index()

In [ ]:
p_values = reg.p_values
p_values = np.append(np.nan, np.array(p_values))
summary_table['p_values'] = p_values

In [ ]:
summary_table

### PD Model Validation

In [ ]:
pred = reg.model.predict_proba(X_test)

In [ ]:
pred

In [ ]:
pred = pred[:][:, 1]

In [ ]:
pred

In [ ]:
y_test_temp = y_test

In [ ]:
y_test_temp.reset_index(drop=True, inplace=True)

In [ ]:
df_actual_predicted_probs = pd.concat([y_test_temp, pd.DataFrame(pred)], axis=1)

In [ ]:
df_actual_predicted_probs.shape

In [ ]:
df_actual_predicted_probs.columns = ['y_test', 'pred_proba']

In [ ]:
df_actual_predicted_probs.index = y_test.index

In [ ]:
df_actual_predicted_probs.head()

In [ ]:
tr = 0.5
df_actual_predicted_probs['pred'] = np.where(df_actual_predicted_probs['pred_proba'] > tr, 1, 0)

In [ ]:
df_actual_predicted_probs.head()

In [ ]:
pd.DataFrame(metrics.confusion_matrix(df_actual_predicted_probs['y_test'], df_actual_predicted_probs['pred']), 
             index = ['Actual: False', 'Actual: True'],
             columns = ['Pred: False', 'Pred: True'])

In [ ]:
print(metrics.classification_report(df_actual_predicted_probs['y_test'], df_actual_predicted_probs['pred']))

#### Accuracy Under the curve

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
roc_curve(df_actual_predicted_probs['y_test'], df_actual_predicted_probs['pred_proba'])

In [ ]:
fpr, tpr, thresholds = roc_curve(df_actual_predicted_probs['y_test'], df_actual_predicted_probs['pred_proba'])

In [ ]:
plt.plot(fpr, tpr)
plt.plot(fpr, fpr, linestyle = '--', color='k')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.xlabel('ROC curve');

In [ ]:
AUROC = roc_auc_score(df_actual_predicted_probs['y_test'], df_actual_predicted_probs['pred_proba'])

In [ ]:
AUROC

#### Gini coefficient & Kolmogorov-Smirnov coefficient

**Gini coefficient**: Measure of the inequality between the rich and poor individuals in an economy, In our case, it'll be used to measure the inequality between the good borrowers and bad borrowers in a population.
Measured by plotting the cummulative proportion of defaulted borrowers(y) as a fuction of the cummulative proportion of defaulted borrowers(x).<br> 
The borrowers must be in ascending order, by estimated probability of being good from the PD model. The Gini coefficient is the percentage of area above the secondary diagonal line, enclosed between the concave curve and the secondary diagonal line. The greater the curve, the better the model.

**Kolmogorov-Smirnov coefficient**: Shows to what extent the model separates the actual good borrowers from the actual bad borrowers. It is the maximum difference between the cummulative distributions of good and bad borrowers. The greater this difference, the better the model

In [ ]:
#sort df as a prerequisite for measuring the gini coefficient.
df_actual_predicted_probs = df_actual_predicted_probs.sort_values('pred_proba')
df_actual_predicted_probs

In [ ]:
df_actual_predicted_probs = df_actual_predicted_probs.reset_index()

In [ ]:
df_actual_predicted_probs

In [ ]:
df_actual_predicted_probs['Cummulative_N_Population'] = df_actual_predicted_probs.index + 1
df_actual_predicted_probs['Cummulative_N_Good'] = df_actual_predicted_probs['y_test'].cumsum()
df_actual_predicted_probs['Cummulative_N_Bad'] = df_actual_predicted_probs['Cummulative_N_Population'] - df_actual_predicted_probs['y_test'].cumsum()

In [ ]:
df_actual_predicted_probs

In [ ]:
df_actual_predicted_probs['Cummulative_Perc_Population'] = df_actual_predicted_probs['Cummulative_N_Population'] / (df_actual_predicted_probs.shape[0])
df_actual_predicted_probs['Cummulative_Perc_Good'] = df_actual_predicted_probs['Cummulative_N_Good'] / df_actual_predicted_probs['y_test'].sum()
df_actual_predicted_probs['Cummulative_Perc_Bad'] = df_actual_predicted_probs['Cummulative_N_Bad'] / (df_actual_predicted_probs.shape[0]- df_actual_predicted_probs['y_test'].sum())

In [ ]:
df_actual_predicted_probs

In [ ]:
plt.plot(df_actual_predicted_probs['Cummulative_Perc_Population'], df_actual_predicted_probs['Cummulative_Perc_Bad'])
plt.plot(df_actual_predicted_probs['Cummulative_Perc_Population'], df_actual_predicted_probs['Cummulative_Perc_Population'], linestyle='--', color='k')
plt.xlabel('Cummulative % Population')
plt.ylabel('Cummulative % Bad')
plt.xlabel('Gini');

In [ ]:
Gini = AUROC * 2 - 1
Gini

In [ ]:
plt.plot(df_actual_predicted_probs['pred_proba'], df_actual_predicted_probs['Cummulative_Perc_Bad'], color='r')
plt.plot(df_actual_predicted_probs['pred_proba'], df_actual_predicted_probs['Cummulative_Perc_Good'], color='b')
plt.xlabel('Estimated Probability for being good')
plt.ylabel('Cummulative %')
plt.xlabel('Kolmogorov-Smirnov');

In [ ]:
KS = max(df_actual_predicted_probs['Cummulative_Perc_Bad'] - df_actual_predicted_probs['Cummulative_Perc_Good'])
KS

In [ ]:
import pickle

pickle.dump(reg, open('model/pd_model.pkl', 'wb'))

### Creating a ScoreCard

In [ ]:
summary_table

In [ ]:
reference_categories

In [ ]:
df_ref_categories = pd.DataFrame(reference_categories, columns=['Feature name'])
df_ref_categories['Coefficients'] = 0
df_ref_categories['p_values'] = np.nan

In [ ]:
df_scorecard = pd.concat([summary_table, df_ref_categories])
df_scorecard = df_scorecard.reset_index()
df_scorecard

In [ ]:
df_scorecard['Original feature name'] = df_scorecard['Feature name'].str.split(':').str[0]

In [ ]:
df_scorecard

In [ ]:
min_score = 300
max_score = 850

In [ ]:
df_scorecard.groupby('Original feature name')['Coefficients'].min()

In [ ]:
df_scorecard.groupby('Original feature name')['Coefficients'].max()

In [ ]:
min_sum_coef = df_scorecard.groupby('Original feature name')['Coefficients'].min().sum()
min_sum_coef

In [ ]:
max_sum_coef = df_scorecard.groupby('Original feature name')['Coefficients'].max().sum()
max_sum_coef

In [ ]:
df_scorecard['Score - Calculation'] = df_scorecard['Coefficients'] * (max_score - min_score)/ (max_sum_coef - min_sum_coef)
df_scorecard

In [ ]:
#CALCULATING SCORE FOR THE INTERCEPT
df_scorecard['Score - Calculation'][0] = ((df_scorecard['Coefficients'][0] - min_sum_coef) / (max_sum_coef - min_sum_coef)) * (max_score - min_score) + min_score
df_scorecard

In [ ]:
df_scorecard['Score - Preliminary'] = df_scorecard['Score - Calculation'].round()
df_scorecard

In [ ]:
min_score_prel = df_scorecard.groupby('Original feature name')['Score - Preliminary'].min().sum()
min_score_prel

In [ ]:
max_score_prel = df_scorecard.groupby('Original feature name')['Score - Preliminary'].max().sum()
max_score_prel

### Calculating Credit Score

In [ ]:
X_train_with_ref_cat.head()

In [ ]:
df_scorecard

In [ ]:
X_test_with_ref_cat_w_intercept = X_test_with_ref_cat

In [ ]:
X_test_with_ref_cat_w_intercept.insert(0, 'Intercept', 1)

In [ ]:
X_test_with_ref_cat_w_intercept.head()

In [ ]:
X_test_with_ref_cat_w_intercept = X_test_with_ref_cat_w_intercept[df_scorecard['Feature name'].values]

In [ ]:
X_test_with_ref_cat_w_intercept.head()

In [ ]:
scorecard_scores = df_scorecard['Score - Preliminary']

In [ ]:
X_test_with_ref_cat_w_intercept.shape, scorecard_scores.shape

In [ ]:
scorecard_scores = scorecard_scores.values.reshape(92, 1)

In [ ]:
scorecard_scores.shape

In [ ]:
scorecard_scores

In [ ]:
#summing the multiplication of the two arrays
y_scores = X_test_with_ref_cat_w_intercept.dot(scorecard_scores)

In [ ]:
y_scores

In [ ]:
y_scores[0].values.min()

### Getting PD from Credit Score

In [ ]:
sum_coef_from_score = ((y_scores - min_score) / (max_score - min_score)) * (max_sum_coef - min_sum_coef) + min_sum_coef

In [ ]:
pred_proba_from_score = np.exp(sum_coef_from_score) / (np.exp(sum_coef_from_score) + 1)
pred_proba_from_score.head()

In [ ]:
pred[0:5]

### Setting a cut-offs

In [ ]:
plt.plot(fpr, tpr)
plt.plot(fpr, fpr, linestyle = '--', color='k')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.xlabel('ROC curve');

In [ ]:
thresholds

In [ ]:
df_cutoffs = pd.concat([pd.DataFrame(thresholds), pd.DataFrame(fpr), pd.DataFrame(tpr)], axis=1)

In [ ]:
df_cutoffs.columns = ['thresholds', 'fpr', 'tpr']

In [ ]:
#the roc curve method sets the first threshold where nothing is being predicted to the maximum probability score from the supplied data + 1
df_cutoffs.head()

In [ ]:
df_cutoffs['thresholds'][0] = 1-1/np.power(10,16)

In [ ]:
df_cutoffs['score'] = ((np.log(df_cutoffs['thresholds']/ (1 - df_cutoffs['thresholds'])) - min_sum_coef) * ((max_score - min_score) / (max_sum_coef - min_sum_coef)) + min_score)

In [ ]:
df_cutoffs.head()

In [ ]:
df_cutoffs['score'][0] = max_score

In [ ]:
df_cutoffs

In [ ]:
df_cutoffs.tail()

In [ ]:
def n_approved(p):
    return np.where(df_actual_predicted_probs['pred_proba'] >= p, 1, 0).sum()

In [ ]:
df_cutoffs['N Approved'] = df_cutoffs['thresholds'].apply(n_approved)
df_cutoffs['N Rejected'] = df_actual_predicted_probs['pred_proba'].shape[0] - df_cutoffs['N Approved']
df_cutoffs['Approval Rate'] = df_cutoffs['N Approved']/ df_actual_predicted_probs['pred_proba'].shape[0]
df_cutoffs['Rejection Rate'] = 1 - df_cutoffs['Approval Rate']

In [ ]:
df_cutoffs.head()

In [ ]:
df_cutoffs.tail()

In [ ]:
#if the lowest posiible probability for being good is 90%
df_cutoffs.iloc[5000:6200, :]

### Save Results

In [ ]:
X_train_with_ref_cat.to_csv('loan_data/PD_train_data.csv')
df_scorecard.to_csv('loan_data/df_scorecard_PD.csv')